## TF-IDF Summarization
TF-IDF(Term Frequency-Inverse Document Frequency)是一種統計方法用來評估一個字詞對於一個文集或語料庫中的其中一份檔案的重要度，在文字探勘和資訊檢索是一個常用的加權技術。依照TF-IDF，字詞的重要性隨著它在檔案中出現的次數成正比增加，但同時會隨著它在語料庫中出現的頻率成反比下降。因此，文檔裏常出現的字詞會比語料庫裏常出現和不重要的詞(e.g. 的，是，個)更有重要性。


計算TF-IDF會需要一個語料庫(corpus)，有時稱爲文集或檔案集。在文字探勘裏，語料庫是指大量的文本，通常經過整理還有標記。這個示範用的語料庫只有100個文檔，算非常小——平常的語料庫會大很多(e.g.一百萬以上)，因爲效果也會進步很多。TF-IDF會需要一個語料庫用來跟選擇分析的文章做比較。TF-IDF的公式如下:

    TF(w)     =  (字詞w在文檔裏出現的次數)   /    (文檔裏的字詞的總數)

    IDF(w)    = log_e(語料庫裏的文檔的總數)  /  (語料庫裏包含字詞w的文檔)

    TF-IDF(w) = TF(w) * IDF(w)
    
**例子**: 我們有一個有100個字詞的文章，字詞 "Python" 在檔案裏出現了3次
1. "Python" 的TF分數會是
        TF(Python) = 3 / 100 = 0.03
2. 假如說我們的語料庫有一千萬文檔，而 "Python" 出現在語料庫一千個文檔裏。"Python" 的IDF分數會是
        IDF(Python) = log(10,000,000/1,000) = 4
3. 最後 "Python" 的TF-IDF分數會是
        TF-IDF(Python) = 0.03 * 0.04 = 12

## 爬文
- 爬蘋果新聞的政治新聞
- 把爬下來的100篇新聞下載當語料庫

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def getDetail(url):
    ret   = {}
    try:
        res   = requests.get(url, verify=False)
        soup  = BeautifulSoup(res.text, 'lxml')
        ret['title']   = soup.select_one('h1').text
        ret['content'] = soup.select_one('.ndArticle_margin p').text
        ret['category'] = soup.select('.current')[1].text
    except:
        print(url)
    return ret

In [3]:
## getDetail('https://tw.appledaily.com/new/realtime/20181128/1474858/')

In [4]:
url = 'https://tw.appledaily.com/politics/realtime/{}'

def crawlApple(newsurl):
    news_ary = []
    for i in range(1,2):
        res = requests.get(newsurl.format(i), verify=False)
        soup = BeautifulSoup(res.text, 'lxml')
        for news in soup.select('.rtddt a'):
            news_ary.append(getDetail('https://tw.appledaily.com'+news.get('href')))
    return news_ary

In [5]:
## newsdf = pd.DataFrame(crawlApple(newsurl)) # 把爬下來的文章變成 pandas DataFrame
## newsdf.head(10)

In [6]:
## newsdf.head(100).to_excel('appledaily_politinews100.xlsx') # 只用100個文件, corpus平常會比較大, 比如說10,000,000個文件

In [7]:
news100 = pd.read_excel('appledaily_politinews100.xlsx')
news100.head()

,category,content,title
0,政治,總統蔡英文卸任民進黨主席，昨在臉書po給黨員的一封信，信中說，民進黨會展開一場轟轟烈烈的檢討...,蔡英文要轟轟烈烈檢討民進黨 林濁水酸：民主國家沒聽過
1,政治,民進黨這次九合一選舉慘敗，蔡英文辭去黨主席一職，昨公布「給黨員的一封信」，內容提到，最該改變...,他列民進黨10大豬隊友 「不下架將賠上2020」
2,政治,2018年地方選舉國民黨拿下空前勝利，執政地區從6縣市竄升為15縣市，昨中常會上國民黨提出選...,選戰檢討報告出爐 國民黨：落選者「缺乏對外拓票企圖心」
3,政治,準高雄市長韓國瑜的愛女韓冰昨(28日)受邀上政論節目《54新觀點》，首度與民進黨台北市議員王...,不用跳愛河！韓冰邀替高雄拚觀光 王世堅靦腆：可以廢物利用
4,政治,點我直接觀看 被操弄的民調？《蘋果》數據解密,【民調解密3】林佳龍敗在紫爆 盧秀燕跌破眼鏡大勝


## 清理文集並創建語料庫
- 清理爬下來文集（刪除標點符號，數字，和停止詞）
- 設定/創造詞庫
- 創造語料庫，然後把它轉換成電腦能分析的格式

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import jieba
import jieba.analyse

In [9]:
text = news100['content']

# 設定適合的標點符號和停止詞
punctuation = [' ','；','，','。','！','：','「','」','｣','…','、','？','【','】','.',':','?',';','!','~','`','+','-','<','>','/','[',']','{','}',"'",'"',     '\xa0','%','）','（','\u3000','\u200b','\t','／']
stop = requests.get('https://raw.githubusercontent.com/tomlinNTUB/Machine-Learning/master/data/%E5%81%9C%E7%94%A8%E8%A9%9E-%E7%B9%81%E9%AB%94%E4%B8%AD%E6%96%87.txt', verify = False)
stopwords = stop.text.split('\r\n')
addedStops = ['httpswwwfacebookcomappledailytw','nannan','好','已','最']

# 刪除標點符號和數字
cleanText = []
for doc in text:
    onedoc=''
    for char in str(doc).replace('\n',''):
        if char not in punctuation and not char.isdigit() :#and char not in stopwords
            onedoc += char
    cleanText.append(onedoc)

# 設定詞庫和加詞
jieba.set_dictionary('dict.txt.big.txt') # 應用繁體詞庫
jieba.add_word('蔡英文') # 加新詞進入詞庫
jieba.add_word('韓國瑜')
jieba.add_word('柯P')
jieba.add_word('柯文哲')
jieba.add_word('邱豐光')
jieba.add_word('民進黨')
jieba.add_word('國民黨')

# 創造語料庫，刪除停止詞
corpus = [] # 語料庫
for cleanDoc in cleanText:
    textList = jieba.lcut(cleanDoc, cut_all=False)
    noBrakesList = []
    for phrase in textList:
        if phrase not in stopwords and phrase not in addedStops:
            noBrakesList.append(phrase)
    corpus.append(' '.join(noBrakesList))

corpus

D:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
Building prefix dict from C:\Users\YB0002777\Desktop\Learning Python\dict.txt.big.txt ...
Loading model from cache C:\Users\YB0002~1\AppData\Local\Temp\jieba.u018f3a7b0c6b32cba5278025feb505ed.cache
Loading model cost 1.501 seconds.
Prefix dict has been built succesfully.


['總統 蔡英文 卸任 民進黨 主席 昨在 臉書 po 黨員 一封信 信中 說 民進黨 展開 一場 轟轟烈烈 檢討 呼籲 時刻 民進黨 人 大鳴大放 不該 指責 責任 一肩 扛起 同志 之間 難前 立委 林 濁水 質疑 從來沒 聽過 民主 國家 政黨 轟轟烈烈 檢討 黨務 林 濁水 臉書 說 立法 院長 蘇嘉全 說 點名 主席 一定 理由 點名 人 好好 想 知道 想 一段時間 蘇嘉全 已經 理由 想 拿出 勇氣 客氣 問一問 林 濁水 認為 無論如何 民主 國家 政治 領袖 先 理想 確定 爭取 位置 理由 費盡心思 想一想 點名 人 理由 總統 說 將來 主席 一定 轟轟烈烈 討論 黨務 民主 國家 轟轟烈烈 討論 施政 戰略 路線 黨政 關係 左翼 政黨 縦 使 黨務 轟轟烈烈 討論 卻 沒聽說過 林 濁水 酸 開天闢地 創舉 成功 歐美 日 各國 肯定 得到 台灣來 取經 請問 蘇嘉全 想 半天 想出 總統 一起 完成 偉大 轟轟烈烈 夢想 何哲欣 台北 報導 想 知道 一定 年前 撞死人 判 緩刑 韓國瑜 下午 出面 說明 選戰 檢討 報告 出爐 國民黨 落選者 缺乏 對外 拓票 企圖心 不用 跳 愛河 韓冰 邀替 高雄 拚 觀光 王世堅 靦腆 廢物利用 這則 新聞 人 陳 其邁 直播 安慰 邁粉 段 宜康 插花 勸進 選 主席 蘇嘉全 曾 參選 副 總統 親英系 背景 看好 接任 黨 主席 動畫 蔡英文 站 第一線 林 濁水 諷賴 清德變 隨身 特助',
 '民進黨 這次 合一 選舉 慘敗 蔡英文 辭去 黨 主席 一職 昨 公布 黨員 一封信 內容 提到 最該 改變 人 責任 一肩 扛起 同志 之間 難前 立委 孫大 千則 說 蔡 總統 辭職 下台 那要 一肩 扛起 所有 責任 這不 一句 空話 孫大千 今天上午 臉書 發文 民進黨 真的 希望 能夠 救亡圖存 力挽狂瀾 建議 短 時間 不論是 勸退 勸辭 撤換 方式 下列 十大 豬 隊友 下架 捨不得 十個 人 保證 賠上 民進黨 年 政權 第一副 總統 陳建仁 理由 水災 時當 南部 鄉親 還泡 水裡 人 卻 金門 玩 好玩 天 第二 總統府 秘書長 陳 菊 理由 高雄市 整到 窮 滿朝 盡是 菊家班 第三 行政院長 賴清德 理由 幹話 王 形象 早已 深植 人心 第四 行政院 發言人 谷辣斯 理由 

In [10]:
# 把語料庫和文章轉換成vector
vect = TfidfVectorizer(token_pattern=r'(?u)\b\w+\b', max_features = 6500)
corpus_tfidf = vect.fit_transform(corpus)

# 語料庫 sparse matrix/vectors
corpus_tfidf

<100x6500 sparse matrix of type '<class 'numpy.float64'>'
	with 17452 stored elements in Compressed Sparse Row format>

In [11]:
# 語料庫最高TF-IDF分數的100個單詞/最重要的單詞
feature_array = np.array(vect.get_feature_names())
tfidf_sorting = np.argsort(corpus_tfidf.toarray()).flatten()[::-1]

n = 100
top_n = feature_array[tfidf_sorting][:n]
top_n

array(['邱豐光', '邱', '開口', '侯', '副', '市長', '目前', '柯文哲', '多以', '邱任', '邱力',
       '人脈廣', '必達', '明升暗降', '搶人', '空穴來風', '商界', '看待', '冀望', '看上', '弟',
       '探詢', '使命', '將於月', '強邱', '調往', '警政', '正式', '王嘉慶', '警', '去年', '警察局',
       '不約而同', '大學', '大戰', '柯侯', '宜', '周刊', '警政署', '人侯友', '副手', '喜歡',
       '各縣市', '私下', '回覆', '來當', '早', '署長', '發現', '一場', '就職', '幕僚', '王',
       '侯友宜', '做好', '柯p', '局長', '綜合', '意願', '議會', '外界', '交換', '前度', '片',
       '邁選', '報導', '效果', '曝光', '挺', '展開', '首長', '協議', '密約', '重要', '找',
       '新北', '君子', '結束', '溝通', '當時', '落幕', '對此', '工作', '選戰', '討論', '決定',
       '台北市', '原本', '參選', '前', '日', '時', '韓國瑜', '月', '當選', '陳', '選舉',
       '知道', '想', '表示'], dtype='<U33')

## Vectorize文章並找出最適合做摘要的句子
- 把文章分成句子
- 把句子轉換成vector，然後計算每個句子的TF-IDF分數
- 找出TF-IDF分數最高的句子用來做長文摘要

In [12]:
import re

# 選擇文章，把文章分成句子
docNum = 31
testDoc = news100['content'][docNum]
testTitle = news100['title'][docNum]

def splitSentences(document):
    #for sent in re.findall(u'[^!?。\.\!\?]+[!?。\.\!\?]?', paragraph, flags=re.U):
    for sent in re.findall(u'[^!?。\!\?]+[!?。\!\?]?', document, flags=re.U):
        yield sent

sentenceDoc = []
for sent in list(splitSentences(testDoc))[:-1]:
    sentenceDoc.append(' '.join(jieba.cut(sent)))

# 原本文章
testDoc

'今年九合一大選投票狀況多，據了解，中選會官方網站在投票日當天曾遭到來自境外大規模分散式阻斷服務（DD0oS）攻擊，所幸在負責網路維護中華電信應變下，未進一步造成影響。中選會副主委陳朝建今證實曾遭駭客攻擊，不過計票網路是封閉系統，並非公開網路，無法由外界接取，因此沒有任何影響，境外網路干擾也並未影響投票進程。行政院發言人Kolas Yotaka（谷辣斯・尤達卡）表示，中選會在選前即做好縝密防護，因此成功防堵攻擊，接下來將針對有形的選務如投票排隊問題，以及無形的選務如資安防護進行檢討。不過，投票日當天開票進程緩慢，台北市選情更創下史上第一次開票長達9小時的紀錄，陳朝建說明，開票時間長是因為投票時間延後，計票不受駭客影響，開票也自然不受此影響。據內部消息指出，投票日當天曾出現從境外進行分散式阻斷服務（DDoS）攻擊，企圖癱瘓中選會網站，不過在中華電信、中選會緊急處理後，阻斷來自國外異常流量，在短時間獲得解決，一般民眾並沒有發現特殊異樣。NCC及中華電信暫未對此做進一步說明。陳朝建表示，受攻擊的是中選會官方網站，當日開票的計票網路是獨立的封閉網路，並非公開網路，無法由外界接取，完全未有任何影響；中華電信團隊也是24小時監控與防護作業，也沒有受到任何影響。陳朝建說，受到攻擊的官網，當日皆有隨時防衛，中華電信團隊24小時監控與防護作業；故當日遭到來自境外大規模分散式阻斷服務（DD0oS）攻擊時，在負責網路維護中華電信的緊急應變下，並沒有受到影響。另外，Kolas表示，中選會的官網在選舉當天上午6時56分至7時01分期間，遭受大規模境外DDoS攻擊，而中選會的系統是封閉系統，防護非常嚴密，因此沒被攻擊，而當因中選會在選前即做好縝密的防護，當天其實沒因網路攻擊發生意外狀況，中選會的資安管理有成功防堵攻擊。Kolas表示，所幸這次在事前有預見可能的危險，中選會的資料庫、網站等都沒遭受損害，而在資訊時代，行政院希望在這波中選會的選務檢討中，不只針對有形被詬病的排隊投票問題，無形的選務如資安防護，也會進行檢討、加強。（林惟崧、鄭鴻達／台北報導）出版：1454更新：1942（新增行政院說法） \xa0想知道更多，一定要看……【獨家】投票日遭境外駭客攻擊\u3000中選會網站險癱瘓看了這則新聞的人，也看了……老里長險勝1票\u3000美女里長候選人要求驗票發祭品囉！\u3000韓總雞排逾千

In [13]:
# 分詞後的文章
sentenceDoc

['今年 九 合一 大選 投票 狀況 多 ， 據 了解 ， 中選會 官方網站 在 投票 日 當天 曾 遭到 來自 境外 大規模 分散式 阻斷 服務 （ DD0oS ） 攻擊 ， 所幸 在 負責 網路 維護 中華電信 應變 下 ， 未 進一步 造成 影響 。',
 '中選會 副 主委 陳 朝 建今 證實 曾 遭 駭客 攻擊 ， 不過 計票 網路 是 封閉系統 ， 並非 公開 網路 ， 無法 由 外界 接取 ， 因此 沒有 任何 影響 ， 境外 網路 干擾 也 並未 影響 投票 進程 。',
 '行政院 發言人 Kolas   Yotaka （ 谷辣斯 ・ 尤 達卡 ） 表示 ， 中選會 在 選前 即 做好 縝密 防護 ， 因此 成功 防堵 攻擊 ， 接下來 將 針對 有形 的 選務 如 投票 排隊 問題 ， 以及 無形 的 選務 如資安 防護 進行 檢討 。',
 '不過 ， 投票 日 當天 開票 進程 緩慢 ， 台北市 選情 更 創下 史上 第一次 開票 長達 9 小時 的 紀錄 ， 陳 朝建 說明 ， 開票 時間 長 是 因為 投票 時間 延後 ， 計票 不 受 駭客 影響 ， 開票 也 自然 不受 此 影響 。',
 '據 內部消息 指出 ， 投票 日 當天 曾 出現 從 境外 進行 分散式 阻斷 服務 （ DDoS ） 攻擊 ， 企圖 癱瘓 中選會 網站 ， 不過 在 中華電信 、 中選會 緊急 處理 後 ， 阻斷 來自 國外 異常 流量 ， 在 短時間 獲得 解決 ， 一般 民眾並 沒有 發現 特殊 異樣 。',
 'NCC 及 中華電信 暫未 對此 做 進一步 說明 。',
 '陳 朝建 表示 ， 受 攻擊 的 是 中選會 官方網站 ， 當日 開票 的 計票 網路 是 獨立 的 封閉 網路 ， 並非 公開 網路 ， 無法 由 外界 接取 ， 完全 未有 任何 影響 ； 中華電信 團隊 也 是 24 小時 監控 與 防護 作業 ， 也 沒有 受到 任何 影響 。',
 '陳 朝建 說 ， 受到 攻擊 的 官網 ， 當日 皆 有 隨時 防衛 ， 中華電信 團隊 24 小時 監控 與 防護 作業 ； 故 當日 遭到 來自 境外 大規模 分散式 阻斷 服務 （ DD0oS ） 攻擊 時 ， 在 負責 網路 維護 中華電信 的 緊急 應變 下 ， 並 沒有 受到 影

In [14]:
# 把文章轉換成vector
doctor_tfidf = vect.transform(sentenceDoc)
doctor_tfidf

<10x6500 sparse matrix of type '<class 'numpy.float64'>'
	with 263 stored elements in Compressed Sparse Row format>

In [15]:
# 把每個句子的總共TF-IDF分數計算好
def tfidf_sum(doc_array):
    sumlst = []
    for sentence in doc_array.toarray():
        sumlst.append(sum(sentence))
    return sumlst
tfidf_sum(doctor_tfidf)

[5.320901241812664,
 4.223532014145172,
 4.582918038414828,
 4.7476349631144235,
 5.331435489581073,
 2.557473534332187,
 4.666741832240017,
 5.151569570930698,
 4.850851288291918,
 5.385329498460101]

## 長文摘要

In [16]:
# 原本文章列爲句子
index = 0
print('標題: ' + '《' + testTitle + '》')
for sentence in sentenceDoc:
    print(str(index) + ' ', sentence.replace(' ', '').strip())
    index += 1

標題: 《中選會證實投票日遭駭客攻擊　行政院：成功防堵》
0  今年九合一大選投票狀況多，據了解，中選會官方網站在投票日當天曾遭到來自境外大規模分散式阻斷服務（DD0oS）攻擊，所幸在負責網路維護中華電信應變下，未進一步造成影響。
1  中選會副主委陳朝建今證實曾遭駭客攻擊，不過計票網路是封閉系統，並非公開網路，無法由外界接取，因此沒有任何影響，境外網路干擾也並未影響投票進程。
2  行政院發言人KolasYotaka（谷辣斯・尤達卡）表示，中選會在選前即做好縝密防護，因此成功防堵攻擊，接下來將針對有形的選務如投票排隊問題，以及無形的選務如資安防護進行檢討。
3  不過，投票日當天開票進程緩慢，台北市選情更創下史上第一次開票長達9小時的紀錄，陳朝建說明，開票時間長是因為投票時間延後，計票不受駭客影響，開票也自然不受此影響。
4  據內部消息指出，投票日當天曾出現從境外進行分散式阻斷服務（DDoS）攻擊，企圖癱瘓中選會網站，不過在中華電信、中選會緊急處理後，阻斷來自國外異常流量，在短時間獲得解決，一般民眾並沒有發現特殊異樣。
5  NCC及中華電信暫未對此做進一步說明。
6  陳朝建表示，受攻擊的是中選會官方網站，當日開票的計票網路是獨立的封閉網路，並非公開網路，無法由外界接取，完全未有任何影響；中華電信團隊也是24小時監控與防護作業，也沒有受到任何影響。
7  陳朝建說，受到攻擊的官網，當日皆有隨時防衛，中華電信團隊24小時監控與防護作業；故當日遭到來自境外大規模分散式阻斷服務（DD0oS）攻擊時，在負責網路維護中華電信的緊急應變下，並沒有受到影響。
8  另外，Kolas表示，中選會的官網在選舉當天上午6時56分至7時01分期間，遭受大規模境外DDoS攻擊，而中選會的系統是封閉系統，防護非常嚴密，因此沒被攻擊，而當因中選會在選前即做好縝密的防護，當天其實沒因網路攻擊發生意外狀況，中選會的資安管理有成功防堵攻擊。
9  Kolas表示，所幸這次在事前有預見可能的危險，中選會的資料庫、網站等都沒遭受損害，而在資訊時代，行政院希望在這波中選會的選務檢討中，不只針對有形被詬病的排隊投票問題，無形的選務如資安防護，也會進行檢討、加強。


In [17]:
# 選擇摘要的長度
sentCount = 3

# 找出文章TF-IDF分數最高的句子做摘要
max_tfidf = sorted(range(len(tfidf_sum(doctor_tfidf))), key=lambda i: tfidf_sum(doctor_tfidf)[i], reverse=True)[:sentCount]
print('標題: ' + '《' + testTitle + '》')
for sentence in sorted(max_tfidf):
    print('Sentence ' + str(sentence) + ' | TF-IDF Score: ' + str(tfidf_sum(doctor_tfidf)[sentence]) + '\n', sentenceDoc[sentence].replace(' ','').strip())

標題: 《中選會證實投票日遭駭客攻擊　行政院：成功防堵》
Sentence 0 | TF-IDF Score: 5.320901241812664
 今年九合一大選投票狀況多，據了解，中選會官方網站在投票日當天曾遭到來自境外大規模分散式阻斷服務（DD0oS）攻擊，所幸在負責網路維護中華電信應變下，未進一步造成影響。
Sentence 4 | TF-IDF Score: 5.331435489581073
 據內部消息指出，投票日當天曾出現從境外進行分散式阻斷服務（DDoS）攻擊，企圖癱瘓中選會網站，不過在中華電信、中選會緊急處理後，阻斷來自國外異常流量，在短時間獲得解決，一般民眾並沒有發現特殊異樣。
Sentence 9 | TF-IDF Score: 5.385329498460101
 Kolas表示，所幸這次在事前有預見可能的危險，中選會的資料庫、網站等都沒遭受損害，而在資訊時代，行政院希望在這波中選會的選務檢討中，不只針對有形被詬病的排隊投票問題，無形的選務如資安防護，也會進行檢討、加強。


**會提高摘要效果的改進**
1. 更進階的方法計算句子的重要性（不僅把一個句子的TF-IDF分數全加起來）
    - 只算每個句子的名詞TF-IDF分數
    - 提高在標題裏的詞的TF-IDF分數
    - 依照句子在文章裏的位置加分
2. 更好的分詞/分句子，詞庫，和語料庫會有更好的效果/摘要
    - 語料庫越大，輸出的效果會越好 （語料庫的文章如果有10,000,000個 vs. 100個)
    - 詞庫加入更多名詞/分詞更準確也會幫助效果
    - 把文章分成句子時，有時候文章的含義會改變
3. 利用同義詞加強詞庫